In [1]:
from stellargraph import StellarGraph
import pandas as pd
from torch_geometric.datasets import TUDataset

2025-06-09 19:17:43.744681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749471463.986619  117035 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749471464.054108  117035 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749471464.702165  117035 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749471464.702225  117035 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749471464.702228  117035 computation_placer.cc:177] computation placer alr

In [2]:
from IPython.display import display, HTML

In [3]:
dataset = TUDataset(root='/tmp/PROTEINS', name='PROTEINS')

In [4]:
dataset[0]

Data(edge_index=[2, 162], x=[42, 3], y=[1])

In [5]:
dataset

PROTEINS(1113)

In [6]:
stellar_graphs = []
labels = []

for data in dataset:
    # Ekstrak fitur node
    node_features = pd.DataFrame(data.x.numpy())
    
    # Edge index: bentuk (2, num_edges), convert jadi DataFrame
    edges = pd.DataFrame(data.edge_index.T.numpy(), columns=["source", "target"])

    # Buat StellarGraph
    sg = StellarGraph(nodes=node_features, edges=edges)
    stellar_graphs.append(sg)

    # Simpan label
    labels.append(int(data.y.item()))

# Konversi label ke Pandas Series agar mirip StellarGraph style
graph_labels = pd.Series(labels)

In [7]:
adjs = [graph.to_adjacency_matrix() for graph in stellar_graphs]

In [8]:
labels = graph_labels.to_numpy(dtype=int)

In [9]:
import numpy as np
import networkx as nx

metrics = []
for adj in adjs:
    # Use from_numpy_array instead of the deprecated from_numpy_matrix
    G = nx.from_numpy_array(adj)
    
    # Calculate network metrics
    num_edges = G.number_of_edges()
    cc = nx.average_clustering(G)  # Average clustering coefficient
    eff = nx.global_efficiency(G)  # Global efficiency
    
    metrics.append([num_edges, cc, eff])

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(metrics, labels, test_size=0.2, random_state=42)


In [11]:
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

clf = svm.SVC()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.7309417040358744
Precision: 0.7123287671232876
Recall: 0.5714285714285714
F1-score: 0.6341463414634146
